# PSIO – Laboratorium 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, exposure, io, color

# Pomocnicze: zapewnij uint8 [0..255] z ewentualnego float [0..1]
def to_uint8(img):
    if img.dtype == np.uint8:
        return img
    img = np.clip(img, 0, 1)
    return (img * 255).astype(np.uint8)

def hist_counts(img_u8, bins, rng=(0, 256)):
    h, _ = np.histogram(img_u8.ravel(), bins=bins, range=rng)
    return h

def show_side_by_side(img_left, title_left, img_right, title_right, cmap='gray', figsize=(10,4)):
    fig, ax = plt.subplots(1, 2, figsize=figsize)
    ax[0].imshow(img_left, cmap=cmap)
    ax[0].set_title(title_left)
    ax[0].axis('off')
    ax[1].imshow(img_right, cmap=cmap)
    ax[1].set_title(title_right)
    ax[1].axis('off')
    plt.tight_layout()
    plt.show()

## Zadanie 1 – histogramy obrazu `camera`

In [ ]:
# 1a) Histogram (256 kubełków) + obraz
cam = data.camera()  # uint8
cam_u8 = to_uint8(cam)
h256 = hist_counts(cam_u8, bins=256, rng=(0, 256))

fig, ax = plt.subplots(1, 2, figsize=(12,4))
ax[0].imshow(cam_u8, cmap='gray')
ax[0].set_title('Obraz: camera')
ax[0].axis('off')
ax[1].plot(h256, color='black')
ax[1].set_title('Histogram (256 kubełków) – linia')
ax[1].set_xlim(0, 255)
ax[1].set_xlabel('Wartość jasności')
ax[1].set_ylabel('Liczność')
plt.tight_layout(); plt.show()

# 1b) Histogram (8 kubełków) – słupki
h8, edges = np.histogram(cam_u8.ravel(), bins=8, range=(0, 256))
centers = 0.5*(edges[:-1] + edges[1:])
plt.figure(figsize=(6,4))
plt.bar(centers, h8, width=(edges[1]-edges[0])*0.9, color='gray', edgecolor='black')
plt.title('Histogram (8 kubełków) – słupki')
plt.xlabel('Wartość jasności')
plt.ylabel('Liczność')
plt.xlim(0, 255)
plt.tight_layout(); plt.show()

# 1c) Histogram skumulowany – linia
cum = np.cumsum(h256)
plt.figure(figsize=(6,4))
plt.plot(cum, color='blue')
plt.title('Histogram skumulowany')
plt.xlabel('Wartość jasności')
plt.ylabel('Suma liczności')
plt.xlim(0, 255)
plt.tight_layout(); plt.show()

## Zadanie 2 – rozciąganie kontrastu i gamma

In [ ]:
# 2a) Wybór progów a, b na podstawie histogramu i rescale_intensity
# Podpowiedź automatyczna (percentyle), ale wartości można nadpisać ręcznie.
p_low, p_high = np.percentile(cam_u8, [1, 99])
a, b = int(p_low), int(p_high)
# a, b = 30, 210  # <- można odkomentować i dobrać ręcznie

cam_rescaled = exposure.rescale_intensity(cam_u8, in_range=(a, b), out_range=(0, 255)).astype(np.uint8)
h_res = hist_counts(cam_rescaled, bins=256, rng=(0, 256))

show_side_by_side(cam_u8, 'camera – oryginał', cam_rescaled, f'camera – rescale_intensity (a={a}, b={b})')
plt.figure(figsize=(10,4))
plt.plot(h256, label='przed', color='gray')
plt.plot(h_res, label='po', color='red')
plt.title('Histogram: przed/po reskalowaniu intensywności')
plt.xlabel('Wartość jasności'); plt.ylabel('Liczność')
plt.xlim(0, 255); plt.legend(); plt.tight_layout(); plt.show()

# Wyjaśnienie: po rozciąganiu kontrastu część kubełków może mieć 0,
# bo wiele sąsiednich wartości wejściowych klastruje do tych samych wartości wyjściowych
# (kwantyzacja do 256 poziomów), a poza [a,b] wartości ulegają saturacji do 0 lub 255.

In [ ]:
# 2b) Operacje dla obrazu ciemnego: dark_image.png (jeśli brak – tworzymy sztuczny)
paths = ['../resources/dark_image.png', 'resources/dark_image.png', 'dark_image.png']
dark = None
for p in paths:
    try:
        dark = io.imread(p)
        break
    except Exception:
        pass
if dark is None:
    # Fallback: przyciemniamy camera() jako substytut
    dark = to_uint8(exposure.adjust_gamma(cam_u8, gamma=1.8))

dark_u8 = to_uint8(dark if dark.ndim == 2 else color.rgb2gray(dark))
# Dobór a,b (percentyle jako start)
p_low2, p_high2 = np.percentile(dark_u8, [1, 99])
a2, b2 = int(p_low2), int(p_high2)
dark_rescaled = exposure.rescale_intensity(dark_u8, in_range=(a2, b2), out_range=(0, 255)).astype(np.uint8)

# Korekcja gamma – do rozjaśnienia używamy gamma < 1
gamma = 0.6
dark_gamma = to_uint8(exposure.adjust_gamma(dark_u8, gamma=gamma))

# Kombinacja: najpierw gamma, potem reskalowanie
dark_combo = to_uint8(exposure.rescale_intensity(dark_gamma, in_range=(a2, b2), out_range=(0, 255)))

fig, ax = plt.subplots(1, 4, figsize=(16,4))
ax[0].imshow(dark_u8, cmap='gray'); ax[0].set_title('Wejście (ciemny)'); ax[0].axis('off')
ax[1].imshow(dark_rescaled, cmap='gray'); ax[1].set_title(f'Rescale (a={a2}, b={b2})'); ax[1].axis('off')
ax[2].imshow(dark_gamma, cmap='gray'); ax[2].set_title(f'Gamma (γ={gamma})'); ax[2].axis('off')
ax[3].imshow(dark_combo, cmap='gray'); ax[3].set_title('Gamma → Rescale'); ax[3].axis('off')
plt.tight_layout(); plt.show()

## Zadanie 3 – histogramy RGB (`chelsea` lub `coffee`)

In [ ]:
rgb = data.chelsea()  # kolorowy
rgb_u8 = to_uint8(rgb)
R, G, B = rgb_u8[...,0], rgb_u8[...,1], rgb_u8[...,2]

# Wykres liniowy: 3 kanały na jednej figurze
bins = 256; rng = (0, 256)
hR = hist_counts(R, bins, rng); hG = hist_counts(G, bins, rng); hB = hist_counts(B, bins, rng)
plt.figure(figsize=(8,4))
plt.plot(hR, color='r', label='R')
plt.plot(hG, color='g', label='G')
plt.plot(hB, color='b', label='B')
plt.title('Histogramy RGB – wykres liniowy')
plt.xlabel('Wartość'); plt.ylabel('Liczność'); plt.xlim(0,255); plt.legend();
plt.tight_layout(); plt.show()

# Wykres słupkowy: 3 subplots na jednej figurze
fig, ax = plt.subplots(1, 3, figsize=(12,3), sharey=True)
edges = np.linspace(0, 256, 33)  # 32 kubełki dla czytelności
hR_b, e = np.histogram(R.ravel(), bins=edges); centers = 0.5*(e[:-1]+e[1:])
hG_b, _ = np.histogram(G.ravel(), bins=edges)
hB_b, _ = np.histogram(B.ravel(), bins=edges)
ax[0].bar(centers, hR_b, width=(edges[1]-edges[0])*0.9, color='r'); ax[0].set_title('R – słupki'); ax[0].set_xlim(0,255)
ax[1].bar(centers, hG_b, width=(edges[1]-edges[0])*0.9, color='g'); ax[1].set_title('G – słupki'); ax[1].set_xlim(0,255)
ax[2].bar(centers, hB_b, width=(edges[1]-edges[0])*0.9, color='b'); ax[2].set_title('B – słupki'); ax[2].set_xlim(0,255)
for a in ax: a.set_xlabel('Wartość')
ax[0].set_ylabel('Liczność')
plt.tight_layout(); plt.show()

## Zadanie 4 – LUT i ręczna zmiana ekspozycji

In [ ]:
# Funkcja liniowa przechodząca przez (128,128) z nachyleniem a i obcięciem do [0,255]
def fun_lin(x, a):
    y = a * (x - 128) + 128
    return int(np.clip(np.rint(y), 0, 255))

def build_lut(a):
    return np.array([fun_lin(x, a) for x in range(256)], dtype=np.uint8)

a_val = 1.2  # nachylenie >1 zwiększa kontrast wokół 128
LUT = build_lut(a_val)

# Podgląd funkcji
x = np.arange(256); y = LUT
plt.figure(figsize=(5,4))
plt.plot(x, y, color='purple')
plt.title(f'Funkcja LUT (a={a_val}) – przechodzi przez (128,128)')
plt.xlabel('Wejście'); plt.ylabel('Wyjście'); plt.xlim(0,255); plt.ylim(0,255)
plt.grid(True, alpha=0.2); plt.tight_layout(); plt.show()

# Zastosowanie LUT do obrazu czarno-białego (camera)
cam_lut = LUT[cam_u8]  # indeksowanie tablicą LUT
show_side_by_side(cam_u8, 'camera – oryginał', cam_lut, f'camera – LUT (a={a_val})')

# Dla porównania druga wartość a (mniejszy kontrast wokół 128)
a_val2 = 0.8; LUT2 = build_lut(a_val2); cam_lut2 = LUT2[cam_u8]
show_side_by_side(cam_lut, f'LUT a={a_val}', cam_lut2, f'LUT a={a_val2}')